# Data Preprocessing

## Import required packages

In [1]:
# import xgboost
import xgboost as xgb
# import packages for hyperparameters tuning
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
# sklearn packages
from sklearn.decomposition import PCA
from sklearn.metrics import roc_auc_score
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.svm import LinearSVC
# miscellaneous
import os.path
from pprint import pprint
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

## Switch to project directory

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')
# %cd drive/MyDrive/COMSW4995_32 AML/AML Final Project/home-credit-default-risk
%cd home-credit-default-risk

Mounted at /content/drive
/content/drive/.shortcut-targets-by-id/1Aq4j1LDPosstk7lY4v_Ck2m1_6P4wfJb/AML Final Project/home-credit-default-risk


## Create directories if necessary

In [3]:
# Check if train_val_data directory exists, if not create directory
if not os.path.exists('train_val_data'):
    os.makedirs('train_val_data')

# Check if transformed directory exists, if not create directory
if not os.path.exists('transformed'):
    os.makedirs('transformed')

## Create labeled test data to evaluate the model by splitting the training dataset.

In [4]:
application_train = pd.read_csv('application_train.csv')
X = application_train.drop(columns=['TARGET'])
y = application_train['TARGET']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
application_train = X_train
application_train['TARGET'] = y_train
application_test = X_val
application_test['TARGET'] = y_val
application_train.to_csv('train_val_data/train.csv', index=False)
application_test.to_csv('train_val_data/test.csv', index=False)

## Transform supplementary datasets

Transform supplementary dataset by creating new features that aggreagate over `SK_ID_CURR` (the primary key of the main dataset (`application`)).

For each of the supplementary datasets in order to merge them with a label, we need to create features such that there is one row per `SK_ID_CURR` which is the primary key of the main dataset..

After research and analyzing the columns and their meanings, I came up with the following three types of transforms:

1. `ohe`: For categorical features, do a one hot encoding of each their unique values and count the number of credits per `SK_ID_CURR` for each unique value
2. `time`: For features that represent time in days, create a column for the first (min), last (max), and then number in past 6 month (>= -182), year (>= -365), and 5 years (>= -365*5)
3. `amount`: For features that represent an amount or a count, take the max, mean, min, and sum.

In order to simplify the assignment of a feature to its type of transform we take advantage of the consistency in naming features and there datatypes:

- We notice that all features that are categorical are of type `object`, while all others are numerical (`int64` or `float64`).
- We notice of the numerical types, the features that represent time have the substring `DAYS` in the name of the feature.

Using this we create the following function to transform the supplementary datasets.

In [5]:
# Define transformation functions
def transform_ohe(df, column):
    # One-hot encode and aggregate counts for each category
    tmp = pd.get_dummies(df[column]).groupby(df['SK_ID_CURR']).sum()
    tmp.columns = [column+'_'+col for col in tmp.columns]
    return tmp

def transform_time(df, column):
    # Calculate time-based aggregates
    grouped = df.groupby('SK_ID_CURR')[column]
    last_applied = grouped.max()
    first_applied = grouped.min()
    num_last_6mo = grouped.apply(lambda x: (x >= -182).sum())
    num_last_year = grouped.apply(lambda x: (x >= -365).sum())
    num_last_5years = grouped.apply(lambda x: (x >= -365*5).sum())
    num_future = grouped.apply(lambda x: (x >= 0).sum())
    return pd.DataFrame({
        f'{column}_LAST': last_applied,
        f'{column}_FIRST': first_applied,
        f'{column}_COUNT_PAST_6MO': num_last_6mo,
        f'{column}_COUNT_PAST_YR': num_last_year,
        f'{column}_COUNT_PAST_5YRS': num_last_5years,
        f'{column}_COUNT_FUTURE': num_future

    })

def transform_amount(df, column):
    # Calculate amount-based aggregates
    grouped = df.groupby('SK_ID_CURR')[column]
    return pd.DataFrame({
        f'{column}_MAX': grouped.max(),
        f'{column}_MEAN': grouped.mean(),
        f'{column}_MIN': grouped.min(),
        f'{column}_SUM': grouped.sum()
    })

def get_transform_dict(df):
    numeric_features = df.select_dtypes(include=['int64', 'float64']).columns
    categorical_features = df.select_dtypes(include=['object']).columns

    # remove any column names that contain '_ID_' in it
    numerical_features = [col for col in numeric_features if '_ID_' not in col]

    transform_dict = dict()
    for col in numeric_features:
        if 'DAYS' in col:
            transform_dict[col] = 'time'
        else:
            transform_dict[col] = 'amount'
    for col in categorical_features:
        transform_dict[col] = 'ohe'
    return transform_dict

# Apply transformations based on the dictionary
def transform_df(file, df):
    if df is None:
        df = pd.read_csv(file)
    transform_dict = get_transform_dict(df)
    result = []
    for column, transform_type in transform_dict.items():
        if transform_type == 'ohe':
            result.append(transform_ohe(df, column))
        elif transform_type == 'time':
            result.append(transform_time(df, column))
        elif transform_type == 'amount':
            result.append(transform_amount(df, column))
    # Concatenate all results on SK_ID_CURR to form the final dataframe
    df_transformed = pd.concat(result, axis=1).reset_index()
    name = file.split('.')[0]
    df_transformed.to_csv('transformed/'+name+'_transformed.csv', index=False)
    return df_transformed

Loop over supplementary datasets and transform them, saving a new .csv in the *transformed* folder.

In [6]:
files = ['previous_application.csv','POS_CASH_balance.csv','installments_payments.csv','credit_card_balance.csv']
for file in files:
    if not os.path.isfile('transformed/'+file.split('.')[0]+'_transformed.csv'):
        df = transform_df(file, None)

Note that *bureau.csv* and *bureau_balance.csv* are not included above, that is because *bureau_balance.csv* does not have the `SK_ID_CURR`, and thus first we will have to create new features that aggreagate over `SK_ID_BUREAU` (the primary key of `bureau`), and then merge it into `bureau` before doing the final transformation of `bureau`.

In order to merge `bureau_balance` into `bureau`, while maintaining `SK_ID_BUREAU` a primary key in `bureau`, we have to make the `SK_ID_BUREAU` in `bureau_balance` unique without losing all the data stored in that table. Fortunately, there are only two other columns, which we can group by and compute new features that have `SK_ID_BUREAU` unique and which we will then merge into `bureau`.

After analyzing the features in `bureau_balance`, I came up with 4 new features:
- MAX_NUM_MONTHS_PAST_DUE: Maximum number of months past due at any particular time for this past credit
- MOST_RECENT_STATUS: Most recent status of this past credit
- TOTAL_NUM_MONTHS_PAST_DUE: Total number of months past due throughout the duration of this credit
- AVG_MONTHS_PAST_DUE: Average number of months past due throughout the duration of this credit

Note that these values are not exact since there are some monthly updates that have unknown statuses, which we ignore here. They are however good enough estimates.

In [7]:
# Load in datasets
bureau = pd.read_csv('bureau.csv')
bureau_balance = pd.read_csv('bureau_balance.csv')
# Remove all rows with STATUS == 'X' or 'C'
bureau_balance_new = bureau_balance.copy()
bureau_balance_new = bureau_balance_new[bureau_balance_new['STATUS'] != 'X']
bureau_balance_new = bureau_balance_new[bureau_balance_new['STATUS'] != 'C']
# Convert the STATUS column to int
bureau_balance_new['STATUS'] = bureau_balance_new['STATUS'].astype(int)
# Create new DataFrame bureau_balance_unique that has one column called
# 'SK_ID_BUREAU' which is the unique of bureau_balance_new['SK_ID_BUREAU']
# Create new column MAX_NUM_MONTHS_PAST_DUE which is the max STATUS in bureau_balance_new
bureau_balance_unique = bureau_balance_new.groupby('SK_ID_BUREAU').agg(
    MAX_NUM_MONTHS_PAST_DUE = ('STATUS', 'max')
)
# Create new column MOST_RECENT_STATUS which is the STATUS associated with the most recent column in bureau_balance_new (where MONTHS_BALANCE is max)
bureau_balance_unique['MOST_RECENT_STATUS'] = bureau_balance_new.sort_values('MONTHS_BALANCE', ascending=False).groupby('SK_ID_BUREAU').first()['STATUS']
# Edit bureau_balance_new STATUS column by converting any STATUS greater than 0 to 1
bureau_balance_new.loc[bureau_balance_new['STATUS'] > 0,'STATUS'] = 1
# Create new column TOTAL_NUM_MONTHS_PAST_DUE which is the sum of the STATUS in bureau_balance_new
# Create new column AVG_NUM_MONTHS_PAST_DUE which is the average of the STATUS in bureau_balance_new
bureau_balance_unique[['TOTAL_NUM_MONTHS_PAST_DUE','AVG_NUM_MONTHS_PAST_DUE']] = bureau_balance_new.groupby('SK_ID_BUREAU').agg(
    TOTAL_NUM_MONTHS_PAST_DUE = ('STATUS', 'sum'),
    AVG_NUM_MONTHS_PAST_DUE = ('STATUS', 'mean')
)
# Reset index
bureau_balance_unique = bureau_balance_unique.reset_index()
# Merge into bureau
bureau = bureau.merge(bureau_balance_unique, on='SK_ID_BUREAU', how='left')

Now we can transform our new `bureau` dataset as we did with the other supplementary datasets above.

In [8]:
df = transform_df('bureau.csv', bureau)